# Minimización de una función con TensorFlow 2.x

Este cuaderno pretende ilustrar las diferencias entre la versión de TensorFlow 2.x y la versión 1.x utilizando el cuaderno introductorio `minimizacion-tf.ipynb`. En el siguiente enlace podrá encontrar una descripción más detallada de las diferencias entre ambas versiones del framwork. 
- [https://www.tensorflow.org/guide/effective_tf2](https://www.tensorflow.org/guide/effective_tf2)

Aunque la versión de TensorFlow 2.0 fue liberada oficialmente en septiembre de 2019, aún se encuentran todavía en la red muchos ejemplos y cursos en línea que utilizan la versión 1.x, por lo que es todavía más didáctico aprender ML con la versión 1. Sin embargo, las mejoras introducidas en la versión 2 permiten un flujo de trabajo más intuitivo y fluido en la mayoría de los casos; y eventualmente, se dejará de dar soporte a la versión 1 de TensorFlow, por lo que es conveniente introducir la utilización de la versión 2.

In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.1.0'

## Función *banana* de Rosenbrock

La función banana se utiliza como *benchmark* en algoritmos de optimización numérica, por la forma tan peculiar de sus curvas de nivel. En este ejemplo, vamos a ilustrar cómo podemos utilizar el algoritmo de optimización denominado Adam para optimizar la función: 
$$ f(x,y) = (1-x)^2 + 100(y-x^2)^2 $$ 

![Banana function](https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Rosenbrock_function.svg/600px-Rosenbrock_function.svg.png)

Ahora vamos a definir la en TensorFlow, quien se encargará de construir el gráfico de computación y nos permitirá obtener sus gradientes a través de **diferenciación automática**. 

## *Eager execution*

Vemos que ahora al crear las variables de nuestra función a optimizar, están evaluadas automáticamente. A este modo de ejecución del gráfico de computación se le conoce como *eager execution* y es una de las características nuevas introducidas en TensorFlow 2.x. Anteriormente, en la versión 1.x era necesario primero crear un gráfico de computación y luego ejecutarlo. 

In [2]:
# tf.reset_default_graph() # esta función ya no existe
# Creamos las variables de la función
x = tf.Variable(tf.random.normal(shape = (1,), seed = 212))
y = tf.Variable(tf.random.normal(shape = (1,), seed = 213))
x, y

(<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.6871292], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([-0.44555563], dtype=float32)>)

Ahora para poder definir nuestra función a optimizar, lo haremos creando una función de Python, en vez de un tensor, como en la versión 1. El decorador `@tf.function` le indica a Python que debe construir un gráfico de computación de TensorFlow y permite que la evaluación de la función sea más eficiente.

In [3]:
# Definición de la función: notar ahora el decorador @tf.function 
@tf.function
def f(x,y):
    f = (1-x)**2 + 100*(y-x**2)**2
    return f

f

## Uso de las sesiones

En la versión 2 de TensorFlow ¡ya no existen las sesiones!, por lo que el siguiente código arroja un error.

In [14]:
# Evaluamos la función en su mínimo global
with tf.Session() as sess:
    minimo_global = sess.run(f, feed_dict={x:(1,), y:(1,)})

print('El mínimo global es %0.2f' % minimo_global)

AttributeError: module 'tensorflow' has no attribute 'Session'

Ahora las sesiones se simplifican utilizando el gráfico de computación definido en las funciones. Ya que TensorFlow 2 opera en modo de *eager execution*, para obtener el valor de $f(x,y)$ basta con hacer una intuitiva llamada de función: 

In [4]:
f(x,y)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([84.31562], dtype=float32)>

In [5]:
f(1, 1)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

## Optimización con gradiente en descenso (*gradient descent*)

Anteriormente, construimos un *loop* de optimización utilizando una sesión de TensorFlow. Ya que ahora las sesiones fueron reemplazadas por funciones, lo conveniente es crear una función que realice un paso del algoritmo de gradiente en descenso. Aunque esto será lo expuesto en el código a continuación, se recomienda que en los problemas de optimización de ML se utilicen las funciones y modelos de la interfaz de alto nivel Keras incluida en TensorFlow 2, por lo que a continuación se mostrará un "equivalente" al código de la versión 1.x. 

Vamos a crear el "nodo optimizador" que implemente el algoritmo Adam. Ahora este algoritmo es parte del subpaquete de Keras. 

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
optimizer

Ahora, creamos una función que devuelve el valor de la función objetivo y los gradientes de la función, esta función nos servirá para darle estos gradientes al nodo optimizador.

In [8]:
@tf.function
def grad(f, x, y):
    with tf.GradientTape() as tape:
        f_val = f(x, y)
    return f_val, tape.gradient(f_val, [x, y])

grad

Acá podemos ver cómo funcionaría un único paso de optimización. Para aplicar un paso del gradiente en descenso, utilizamos el método `apply_gradients` del nodo optimizador, el cual recibe un iterable de los gradientes y las variables respecto a las que fueron computados (las variables de control). 

In [9]:
# Valores iniciales de f y sus gradientes
f_val, grads = grad(f, x, y)
print("Paso: {}, Valor de f: {}".format(optimizer.iterations.numpy(), f_val.numpy()))
# print(grads)

optimizer.apply_gradients(zip(grads, [x, y]))
print("Paso: {}, Valor de f: {}".format(optimizer.iterations.numpy(), f(x, y).numpy()))

Paso: 0, Valor de f: [84.31562]
Paso: 1, Valor de f: [64.36963]


In [16]:
# Ahora creamos un ciclo de optimización, que aplica pasos de optimización 
# sucesivamente durante N iteraciones
N = 2500
for i in range(N):
    # Valores iniciales de f y sus gradientes
    f_val, grads = grad(f, x, y)

    # Aplicamos un paso de gradiente en descenso
    optimizer.apply_gradients(zip(grads, [x, y]))
    
    if i % 100 == 0:
        print("%i\t(%0.2f, %0.2f):\t%0.4f" % (i, x.numpy(), y.numpy(), f_val.numpy()))

0	(0.59, -0.35):	64.3696
100	(0.15, 0.02):	0.7249
200	(0.35, 0.12):	0.4299
300	(0.50, 0.24):	0.2558
400	(0.61, 0.37):	0.1562
500	(0.69, 0.47):	0.0969
600	(0.76, 0.57):	0.0604
700	(0.81, 0.65):	0.0376
800	(0.85, 0.72):	0.0232
900	(0.88, 0.78):	0.0141
1000	(0.91, 0.83):	0.0084
1100	(0.93, 0.86):	0.0049
1200	(0.95, 0.90):	0.0028
1300	(0.96, 0.92):	0.0016
1400	(0.97, 0.94):	0.0008
1500	(0.98, 0.96):	0.0004
1600	(0.99, 0.97):	0.0002
1700	(0.99, 0.98):	0.0001
1800	(0.99, 0.99):	0.0000
1900	(1.00, 0.99):	0.0000
2000	(1.00, 0.99):	0.0000
2100	(1.00, 1.00):	0.0000
2200	(1.00, 1.00):	0.0000
2300	(1.00, 1.00):	0.0000
2400	(1.00, 1.00):	0.0000


Ahora obtenemos los resultados del proceso de optimización directamente en los tensores `x` e `y`, que fueron actualizadas interactivamente durante cada uno de los pasos de optimización:

In [17]:
x

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.9997714], dtype=float32)>

In [18]:
y

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.99954224], dtype=float32)>

## Conclusión 

Como vemos, con la versión 2.x de Tensorflow, la mayoría del proceso de definición de la función quedó más intuitivamente expresado en el código de Python, respecto a la versión 1.x. En general, la optimización y definición de los modelos se hará con la interfaz de alto nivel Keras, por lo que el código acá es solamente con fines ilustrativos, aunque puede ser extendido para proveer mayor flexibilidad o incorporación de flujos de trabajo muy específicos y no definidos en Keras.